In [1]:
import numpy as np

In [2]:
def generate_input(config):
    species = list(set(config.get_chemical_symbols()))    
    sp_idx = {sp:i for i, sp in enumerate(species)}
    x = []
    for atom in config:
        x.append((*atom.position, sp_idx[atom.symbol]))
    x = np.array(x)
    return x


In [3]:
params_dict = {
    'Au':{
        'eta2': 3.197 ,
        'n0': 0.042 ,
        'e0': -3.8 ,
        'lmbda': 4.182 ,
        'v0': 0.348 ,
        'kappa': 3.249 ,
        's0': 1.642,
    },
    'H':{
        'eta2':4.761 ,
        'n0':0.203 ,
        'e0':-2.371 ,
        'lmbda':7.549 ,
        'v0':0.234 ,
        'kappa':8.747 ,
        's0':0.680,
    }
}

def params_dict2array(params_dict):
    nspecies = len(params_dict)
    species = list(params_dict.keys())
    params_array = np.zeros((7*nspecies)).flatten()
    for i, atom in enumerate(species):
        params_array[7*i:7*(i+1)] = [params_dict[atom]['eta2'], 
                           params_dict[atom]['n0'],
                           params_dict[atom]['e0'],
                           params_dict[atom]['lmbda'],
                           params_dict[atom]['v0'],
                           params_dict[atom]['kappa'],
                           params_dict[atom]['s0']]
    return params_array

In [4]:
dq_fcc = [np.sqrt(i) for i in range(1,11)]
bq_fcc = [12, 6, 24, 12, 24, 8, 48, 6, 36, 24]
dq_bcc = [1, 2/np.sqrt(3), np.sqrt(8/3), np.sqrt(11/3), 2, 4/np.sqrt(3), np.sqrt(19/3), np.sqrt(20/3), np.sqrt(8), 3]
bq_bcc = [8, 6, 12, 24, 8, 6, 24, 24, 24, 32]
cutoff = 1.5
beta_fcc = np.cbrt(16*np.pi/3)/np.sqrt(2)
beta_bcc = np.cbrt(3*np.pi**2)
bohr2ang = 0.529177211
eta1 = 0.5/bohr2ang

class EMT:
    def __init__(self, params, ntypes, struct='fcc'):
        self.params = params
        self.ntypes = ntypes
        if struct == 'fcc':
            self.b = bq_fcc
            self.d = dq_fcc
            self.rc = self.b[2]
            self.rr = 2*self.rc/(0.5*(self.d[2]+self.d[3]))
            self.alpha = 9.210340371976184/(self.rr-self.rc)
            self.beta = beta_fcc
        elif struct == 'bcc':
            self.b = bq_bcc
            self.d = dq_bcc
            self.rc = self.b[2]
            self.rr = 4*self.rc/(self.d[2]+self.d[3])
            self.alpha = 9.210340371976184/(self.rr-self.rc)
            self.beta = beta_bcc
        else:
            raise KeyError(f"Unknown struct:{struct}, Please use 'fcc' or 'bcc'.")
        self.initialize()
    
    def initialize(self):
        self.eta2 = np.zeros(self.ntypes)
        self.n0 = np.zeros(self.ntypes)
        self.e0 = np.zeros(self.ntypes)
        self.lmbda = np.zeros(self.ntypes)
        self.v0 = np.zeros(self.ntypes)
        self.kappa = np.zeros(self.ntypes)
        self.s0 = np.zeros(self.ntypes)
        for i in range(self.ntypes):
            self.eta2[i] = params[7*i+0]
            self.n0[i] = params[7*i+1]
            self.e0[i] = params[7*i+2]
            self.lmbda[i] = params[7*i+3]
            self.v0[i] = params[7*i+4]
            self.kappa[i] = params[7*i+5]
            self.s0[i] = params[7*i+6]
        self.gamma1, self.gamma2 = self.calc_gamma()
        self.chi = self.calc_chi()
    
    def calc_gamma(self):
        gamma1 = np.zeros(self.ntypes)
        gamma2 = np.zeros(self.ntypes)
        beta_s0 = self.beta*self.s0
        print(self.beta, beta_s0, self.d[:3]-self.d[2])
        for q in range(3):
            denom = 1+np.exp(self.alpha*(self.d[q]-self.d[2])*beta_s0)
            gamma1 += self.b[q]/self.b[0]/denom*np.exp(-self.eta2*(self.d[q]-1.0)*beta_s0)
            gamma2 += self.b[q]/self.b[0]/denom*np.exp(-self.kappa*(self.d[q]-1.0)*beta_s0)
        return gamma1, gamma2
    
    def calc_chi(self):
        chi = np.zeros((self.ntypes, self.ntypes))
        for a in range(self.ntypes):
            chi[a, a] = 1.0
            for b in range(a, self.ntypes):
                chi[a,b] = self.n0[b]/self.n0[a]*np.exp(-eta1*(self.s0[b]-self.s0[a]))
                chi[b,a] = 1/chi[a,b]
        return chi

    def neutral_sphere_radius(self, x, cell_mat):
        natoms = x.shape[0]        
        cell_imat = np.linalg.inv(cell_mat)
        sigma = np.zeros((natoms, self.ntypes))
        s = np.zeros(natoms)
        r = np.zeros((natoms, natoms))
        for i, xi in enumerate(x):
            a = int(xi[3])
            for j, xj in enumerate(x):
                b = int(xj[3])
                r3temp = xi[:3] - xj[:3] 
                r3temp = np.matmul(cell_imat, r3temp)
                r3temp = r3temp - np.rint(r3temp)
                r3temp = np.matmul(cell_mat, r3temp)

                r[i,j] = np.linalg.norm(r3temp)

                if (r[i,j] > cutoff*self.rc) or (i==j):
                     continue
                
                r3temp = r3temp/r[i,j]
                
                rtemp = np.exp(self.alpha*(r[i,j] - self.rc))
                theta = 1.0/ (1 + rtemp)
                rtemp1 = self.alpha*rtemp*theta
                sigma[i, b] += theta*np.exp(-self.eta2[b]*(r[i,j]-self.beta*self.s0[b]))
        
            sigma[i,:] = 1/self.gamma1[a] * sigma[i,:]
            s[i] = self.s0[a] - 1/self.beta/self.eta2[a] *np.log(np.sum(sigma[i,:]*self.chi[a,:])/(self.b[0]))
        print(sigma)
        return s, r
    

    def energy(self, x, cell_mat):
        s, r = self.neutral_sphere_radius(x, cell_mat)
        print(s)
        Ecorr = 0.0 
        Eref = 0.0
        for i, xi in enumerate(x):
            a = int(xi[3])
            stemp = self.lmbda[a]*(s[i]-self.s0[a])
            Eref += self.e0[a]*((1.0 + stemp)*np.exp(-stemp) - 1.0)
            Ecorr -= -self.b[0]*self.v0[a]*np.exp(-self.kappa[a]*(s[i]-self.s0[a])) 
            for j, xj in enumerate(x):
                b = int(xj[3])
                if (r[i,j] > cutoff*self.rc) or (i==j):
                    continue
                theta = 1.0/(1.0 + np.exp(self.alpha*(r[i,j] - self.rc)))
                if a==b:
                    Ecorr += -self.v0[a]/2/self.gamma2[a] *np.exp(-self.kappa[a]*(r[i,j]/self.beta - self.s0[a]))*theta
                elif a!=b:
                    Ecorr += -self.chi[a,b]*self.v0[a]/self.gamma2[a] *np.exp(-self.kappa[b]*(r[i,j]/self.beta - self.s0[b]))*theta
        E = Eref + Ecorr 
        print(Eref, Ecorr)
        return E
    
    def force(self,):
        pass

In [5]:
from ase.io import read, write
from ase import Atoms
from ase.build import add_adsorbate, fcc111, bulk
a = 1/np.cbrt(3/(16*np.pi))*1.642
# config = bulk('Au', a=a)
# config = config.repeat((3,3,3))
config = fcc111('Au', (3,3,4), a=a, vacuum=10.0)
add_adsorbate(config, 'H', 1.5, 'ontop')

In [6]:
x = generate_input(config)
params = params_dict2array(params_dict)
cell_mat = config.get_cell()
emt = EMT(params, 2)
emt.energy(x, cell_mat)

1.809399790563555 [2.97103446 1.23039186] [-0.73205081 -0.31783725  0.        ]
[[1.47004031e+03 2.45686266e-16]
 [1.10471795e+01 2.39274002e-17]
 [9.19636820e+00 1.89789752e-19]
 [1.51046056e+01 2.39274002e-17]
 [9.01045811e+00 3.17186737e-19]
 [1.31375150e+01 5.30638543e-17]
 [8.58844763e+00 4.19527964e-20]
 [1.12796301e+01 2.11806989e-17]
 [1.46970603e+03 2.16014632e-16]
 [1.47249073e+03 8.62469658e-12]
 [1.39256062e+01 2.50200536e-14]
 [1.03993207e+01 1.36536148e-13]
 [1.62160360e+01 2.50200536e-14]
 [1.20250305e+01 7.69398243e-13]
 [1.63961693e+01 1.72198143e-11]
 [1.22129318e+01 1.79573463e-18]
 [1.55014973e+01 4.76071609e-12]
 [1.47417292e+03 4.76071609e-12]
 [1.47436137e+03 4.79800542e-07]
 [1.57846180e+01 5.74662107e-09]
 [1.25817938e+01 7.72729329e-10]
 [1.34556514e+01 1.35487058e-10]
 [1.19716968e+01 2.47009262e-13]
 [1.76480333e+01 7.13533500e-07]
 [1.30217656e+01 1.60792951e-08]
 [1.55233059e+01 3.63531529e-13]
 [1.47079592e+03 1.04774342e-07]
 [1.46801104e+03 2.74114036e-

2932.9841679846795

In [7]:
emt.gamma1, emt.gamma2

(array([1.01066315, 1.0449969 ]), array([1.00995866, 0.99748816]))